In [1]:
# -----------------------------
# 1. Import Libraries
# -----------------------------
import cv2
import numpy as np
from util import get_parking_spots_bboxes, empty_or_not


C:\Users\avins\AppData\Roaming\Python\Python313\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.1.3 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
# -----------------------------
# 2. Helper Function
# -----------------------------
def calc_diff(im1, im2):
    """Calculate the mean pixel difference between two images."""
    return np.abs(np.mean(im1) - np.mean(im2))


In [3]:
# -----------------------------
# 3. File Paths
# -----------------------------
mask_path = './mask_1920_1080.png'
video_path = './data/parking_1920_1080.mp4'

In [4]:

# -----------------------------
# 4. Load Mask & Video
# -----------------------------
# Load mask (grayscale)
mask = cv2.imread(mask_path, 0)
if mask is None:
    raise FileNotFoundError(f"Mask file not found: {mask_path}")

# Open video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise FileNotFoundError(f"Video file not found or can't be opened: {video_path}")



In [5]:

# -----------------------------
# 5. Detect Parking Spots from Mask
# -----------------------------
connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)
spots = get_parking_spots_bboxes(connected_components)


In [6]:

# -----------------------------
# 6. Initialize Tracking Variables
# -----------------------------
spots_status = [None] * len(spots)  # Occupied/Empty status
diffs = [None] * len(spots)         # Frame differences
previous_frame = None               # Store previous frame
frame_nmr = 0
step = 30                           # Process every 30 frames


In [7]:

# -----------------------------
# 7. Main Loop: Process Video Frames
# -----------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break  # End of video

    # --- Step A: Calculate Differences ---
    if frame_nmr % step == 0 and previous_frame is not None:
        for spot_indx, spot in enumerate(spots):
            x1, y1, w, h = spot
            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]
            prev_crop = previous_frame[y1:y1 + h, x1:x1 + w, :]
            diffs[spot_indx] = calc_diff(spot_crop, prev_crop)

        # Debug: print diffs sorted
        print([diffs[j] for j in np.argsort(diffs)][::-1])

    # --- Step B: Update Spot Status ---
    if frame_nmr % step == 0:
        if previous_frame is None:
            arr_ = range(len(spots))
        else:
            arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.max(diffs) > 0.4]

        for spot_indx in arr_:
            x1, y1, w, h = spots[spot_indx]
            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]
            spots_status[spot_indx] = empty_or_not(spot_crop)

        previous_frame = frame.copy()

    # --- Step C: Draw Parking Spot Boxes ---
    for spot_indx, (x1, y1, w, h) in enumerate(spots):
        color = (0, 255, 0) if spots_status[spot_indx] else (0, 0, 255)
        cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), color, 2)

    # --- Step D: Display Counter ---
    cv2.rectangle(frame, (80, 20), (550, 80), (0, 0, 0), -1)
    cv2.putText(frame, f'Available spots: {sum(spots_status)} / {len(spots_status)}',
                (100, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # --- Step E: Show Frame ---
    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    frame_nmr += 1


# -----------------------------
# 8. Cleanup
# -----------------------------
cap.release()
cv2.destroyAllWindows()


[np.float64(7.591036414565821), np.float64(7.32528180354268), np.float64(7.1615858737298055), np.float64(7.014159586873248), np.float64(6.439095928226351), np.float64(6.401851851851859), np.float64(6.110378634212296), np.float64(6.088122605363992), np.float64(6.0571895424836555), np.float64(5.906423331544119), np.float64(5.861111111111114), np.float64(5.743794769282033), np.float64(5.742295518907213), np.float64(5.624070317782298), np.float64(5.597826086956523), np.float64(5.5755772005772), np.float64(5.574183006535947), np.float64(5.572663139329805), np.float64(5.556302521008391), np.float64(5.540896218557393), np.float64(5.507366035292208), np.float64(5.419623521572547), np.float64(5.369315342328846), np.float64(5.301759834368532), np.float64(5.296618357487915), np.float64(5.263235294117649), np.float64(5.246798029556643), np.float64(5.236392914653791), np.float64(5.215059137098109), np.float64(5.198951994590942), np.float64(5.186162870945481), np.float64(5.165307971014485), np.float